In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import autograd
import torch.nn.functional as F
import csv

images = np.load("./images.npy")
labels = np.load("./labels.npy")
test = np.load("./part_2_test_images.npy")
height = images.shape[1]
width = images.shape[2]
size = height * width
images = (images - images.mean()) / images.std()
data = images.reshape(images.shape[0],size)
data = torch.from_numpy(data).float().cuda()
labels = torch.from_numpy(labels).float().cuda()
test_data = test.reshape(test.shape[0], size)
test_data = (test_data - test_data.mean()) / test_data.std()
test_data = torch.from_numpy(test_data).float().cuda()
batch_size = 1
NUM_OPT_STEPS = 5000
train_seqs = data[0:45000,:]
train_labels = labels[0:45000]
val_seqs = data[45000:,:]
val_labels = labels[45000:]
NUM_CLASSES = 5

In [2]:
class TooSimpleConvNN(torch.nn.Module):
    def __init__(self, chan_1, chan_2, chan_3, chan_4):
        super().__init__()
        # 3x3 convolution that takes in an image with one channel
        # and outputs an image with 8 channels.
        self.conv1 = torch.nn.Conv2d(1, chan_1, kernel_size=3)
        # 3x3 convolution that takes in an image with 8 channels
        # and outputs an image with 16 channels. The output image
        # has approximately half the height and half the width
        # because of the stride of 2.
        self.conv2 = torch.nn.Conv2d(chan_1, chan_2, kernel_size=3, stride=1)
        self.conv3 = torch.nn.Conv2d(chan_2, chan_3, kernel_size=3, stride=1)
        self.conv4 = torch.nn.Conv2d(chan_3, chan_4, kernel_size=3, stride=1)
        # 1x1 convolution that takes in an image with 16 channels and
        # produces an image with 5 channels. Here, the 5 channels
        # will correspond to class scores.
        self.final_conv = torch.nn.Conv2d(chan_4, 5, kernel_size=1)
    def forward(self, x):
        # Convolutions work with images of shape
        # [batch_size, num_channels, height, width]
        x = x.view(-1, height, width).unsqueeze(1)

        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=3, stride=1)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=3, stride=1)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=3, stride=1)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        n, c, h, w = x.size()
        x = F.avg_pool2d(x, kernel_size=[h, w])
        x = self.final_conv(x).view(-1, NUM_CLASSES)
        return x


In [3]:
def train(model, optimizer, batch_size):
#def train(batch_size):
    # model.train() puts our model in train mode, which can require different
    # behavior than eval mode (for example in the case of dropout).
    model.train()
    # i is is a 1-D array with shape [batch_size]
    i = np.random.choice(train_seqs.shape[0], size=batch_size, replace=False)
    i = torch.from_numpy(i).long().cuda()
    x = autograd.Variable(train_seqs[i, :])
    y = autograd.Variable(train_labels[i]).long()
    i.cpu()
    optimizer.zero_grad()
    y_hat_ = model(x)
    loss = F.cross_entropy(y_hat_, y)
    loss.backward()
    optimizer.step()
    return loss.data[0]

In [4]:
def approx_train_accuracy(model):
    i = np.random.choice(train_seqs.shape[0], size=1000, replace=False)
    i = torch.from_numpy(i).long().cuda()
    x = autograd.Variable(train_seqs[i, :])
    y = autograd.Variable(train_labels[i]).long()
    y_hat_ = model(x)
    pred = []
    for j in range(y_hat_.size()[0]):
        logits = y_hat_[j,:].cpu().data.numpy()
        pred.append(np.argmax(logits))
    return accuracy(pred, y.data.cpu().numpy())

In [5]:
def val_accuracy(model):
    x = autograd.Variable(val_seqs)
    y = autograd.Variable(val_labels)
    y_hat_ = model(x)
    pred = []
    for j in range(y_hat_.size()[0]):
        logits = y_hat_[j,:].cpu().data.numpy()
        pred.append(np.argmax(logits))
    return accuracy(pred, y.data.cpu().numpy())

In [6]:
def accuracy(y, y_hat):
    return (y == y_hat).astype(np.float).mean()

In [7]:
def plot(train_accs, val_accs):
    plt.figure(200)
    plt.title('Training Accuracy')
    plt.xlabel('Iteration')
    plt.ylabel('Accuracy')
    plt.plot(train_accs, 'b')
    plt.show()
    plt.figure(300)
    plt.title('Validation Accuracy')
    plt.xlabel('Iteration')
    plt.ylabel('Accuracy')   
    plt.plot(val_accs, 'b')
    plt.show()

In [8]:
def runModel(model, batch_size, NUM_OPT_STEPS, optimizer):
    train_accs, val_accs = [], []
    for i in range(NUM_OPT_STEPS):
        train(model, optimizer, batch_size)
        if i % 100 == 0:
            train_accs.append(approx_train_accuracy(model))
            val_accs.append(val_accuracy(model))
            print("%6d %5.2f %5.2f" % (i, train_accs[-1], val_accs[-1]))
    print(np.mean(val_accs))
    plot(train_accs, val_accs)

In [9]:
layer_1 = 32
layer_2 = 64
layer_3 = 128
layer_4 = 256
batch = 64
rate = 0.001
step = 20000
model = TooSimpleConvNN(layer_1, layer_2, layer_3, layer_4)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=rate)
runModel(model, batch, step, optimizer)

RuntimeError: cuda runtime error (2) : out of memory at d:\projects\pytorch\torch\lib\thc\generic/THCStorage.cu:66

In [ ]:
with open('jzhan127_part2.csv', 'w', newline='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['id', 'label'])
    
    x = autograd.Variable(test_data)
    y_hat_ = model(x)
    for i in range(5000):
        filewriter.writerow([i, np.argmax(torch.max( y_hat_[i,:].data, 0)[1][0])])